<a href="https://colab.research.google.com/github/dvircohen0/Machine-Learning-Algorithms-From-Scratch/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tqdm.notebook as tq
from numpy import loadtxt
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

In [8]:
pima = loadtxt('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv', delimiter=',')
X = pima[:,:-1]
y = pima[:,-1]
y=(y*2)-1
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X) 
x_train,x_test,y_train,y_test = train_test_split(scaled_data,y)

In [11]:
iterr = 4000
C = 0.01

In [12]:
def svm_SGD(x,y,a,C):
    ww = np.ones(len(x[0]))     
    bb = 1        
    iterr = 4000                
    for e in tq.tqdm(range(iterr)):
        for i in range(len(x)):
            val1= np.dot(x[i], ww) - bb
            if (y[i]*val1 < 1):
                ww -= a * ( -y[i]*x[i] + 2*C*ww )
                bb -= a * (y[i])
            else:
                ww -= a * 2*C*ww
                bb -= a * 0
    return ww, bb

In [13]:
def accuracy(predict,actual):
    count=0
    for i in range(len(predict)):
        if predict[i]==actual[i]:
            count+=1
    return(count/len(predict))

In [14]:
start_W= np.ones(len(x_train[0]))  
start_B= 1

W, B = svm_SGD(x_train, y_train, 0.001,C)

result= []
for i  in range(len(x_test)):
    if np.dot(x_test[i], W) - B > 1:
       result.append(1)
    else:
        result.append(-1)

print('accuracy percentage {:.2%}'.format(accuracy(result,y_test)))     


accuracy percentage 73.44%


In [15]:
from sklearn.svm import LinearSVC

SVM =  LinearSVC(max_iter=2000)
SVM.fit(x_train, y_train)

print('SKlearn accuracy percentage {:.2%}'.format(SVM.score(x_test,y_test)))  

SKlearn accuracy percentage 72.40%
